# Exploratory Data Analysis of Last.fm's Mood Tags

The Last.fm Tags dataset is a complementary dataset included as part of the Million Song Dataset. More information can be found [here](https://labrosa.ee.columbia.edu/millionsong/lastfm).

The dataset is available in several different forms including individual json files for each track as well as an sqlite db. Iterating over the json files is cumbersome, so we make use of the sqlite db here.

The goal of this workbook is to explore the tags available in the Last.fm dataset with a special focus on the moods targeted in our project.

In [89]:
from importlib import reload
from pprint import pprint
import pandas as pd

%pprint

Pretty printing has been turned ON


# Getting Started

### Connecting to the DB

In [64]:
from lastfm_labels import LASTFM_TAGS_DB
import sqlite3

conn = sqlite3.connect(LASTFM_TAGS_DB)

### Example Last.fm Tags Queries

These examples are taken from the very helpful [MSD provided example code](https://labrosa.ee.columbia.edu/millionsong/sites/default/files/lastfm/demo_tags_db.py) demonstrating how to work with the last.fm sqlite db.

In [65]:
from lastfm_labels import sanitize

In [66]:
print('************** DEMO 4 **************')
tag = 'Acid Smurfs'
print('We get all tracks for the tag: {0}'.format(tag))
sql = "SELECT tids.tid FROM tid_tag, tids, tags WHERE tids.ROWID=tid_tag.tid AND tid_tag.tag=tags.ROWID AND tags.tag='%s'" % sanitize(tag)
res = conn.execute(sql)
data = res.fetchall()

pprint(data)

************** DEMO 4 **************
We get all tracks for the tag: Acid Smurfs
[('TRDATLJ128F92FC75F',),
 ('TRVTZIF128F42725C1',),
 ('TREPQOO128F9328886',),
 ('TROXKKP128F427886F',)]


In [110]:
print('************** DEMO 5 **************')
print("We get all tags and the number of tracks they're applied to")
sql = "SELECT tags.tag, COUNT(tid_tag.tid) FROM tid_tag, tags WHERE tid_tag.tag=tags.ROWID GROUP BY tags.tag"
res = conn.execute(sql)
data = res.fetchall()
data = sorted(data, key=lambda x: x[1], reverse=True)
print('after sorting...')
for k in range(10):
    print(data[k])
print('...')

************** DEMO 5 **************
We get all tags and the number of tracks they're applied to
after sorting...
('rock', 101071)
('pop', 69159)
('alternative', 55777)
('indie', 48175)
('electronic', 46270)
('female vocalists', 42565)
('favorites', 39921)
('Love', 34901)
('dance', 33618)
('00s', 31432)
...


### Last.fm DB Schema Exploration

In order to use the provided db effectively, we need to know its schema. Here we use sql commands to expose its table structure (credit: https://www.tomordonez.com/get-schema-sqlite-python.html).

It turns out that the schema is incredibly simple.


In [67]:
cur = conn.cursor()

tables = list()
for row in cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall():
    tables.append(row[0])

print('Tables: {0}\n'.format(tables))

for table in tables:
    
    cols = list()
    for row in cur.execute("pragma table_info('{0}')".format(table)).fetchall():
        cols.append(row[1])

    row_count = cur.execute("SELECT Count(*) FROM {0}".format(table)).fetchall()[0][0]
        
    print('Table "{0}"'.format(table))
    print('\tcolumns: {0}'.format(cols))
    print('\trows: {0}'.format(row_count))
    
cur.close()

Tables: ['tags', 'tids', 'tid_tag']

Table "tags"
	columns: ['tag']
	rows: 522366
Table "tids"
	columns: ['tid']
	rows: 505216
Table "tid_tag"
	columns: ['tid', 'tag', 'val']
	rows: 8598630


# Tag EDA

Now that we know the schema and how to connect to and query the db, we can investigate its contents.

In [87]:
reload(lastfm_labels)

<module 'lastfm_labels' from 'C:\\Users\\jcworkma\\Documents\\personal\\gradschool\\berkeley\\W266\\w266-group-project_lyric-mood-classification\\lastfm_labels.py'>

### Number of Tags

In [82]:
from lastfm_labels import query

data = query(conn, "SELECT tag FROM tags")
print('Number of Tags:', len(data))

data = query(conn, "SELECT tag FROM tags LIMIT 10")
print('\nExample Tags:', ([x[0] for x in data]))


Number of Tags: 522366

Example Tags: ['classic rock', 'Progressive rock', 'blues', 'memphis slim', 'pop', '70s', 'Middle of the road', 'Bonjour ca va', 'Tony Levin', 'instrumental']


In [112]:
df = pd.read_sql_query("SELECT tags.tag, COUNT(tid_tag.tid) as count FROM tid_tag, tags WHERE tid_tag.tag=tags.ROWID GROUP BY tags.tag ORDER BY count desc", conn)

In [114]:
import plotly.plotly as py
import plotly.graph_objs as go

histodata = df.head(100)
data = [go.Bar(x=histodata.tag,
            y=histodata.count)]

py.iplot(data, filename='lastfm_tag_100_histo')

ModuleNotFoundError: No module named 'plotly'

### Tag Count Histogram

In [104]:
# credit: https://stackoverflow.com/questions/485409/generating-a-histogram-from-column-values-in-a-database
df = pd.read_sql_query("SELECT tag, COUNT(tag) as count FROM tid_tag GROUP BY tag ORDER BY tag", conn)
print(df.shape)
print(df['count'].max())
df = df.sort_values('count')
print(df.head())


(522366, 2)
101071
           tag  count
522365  522366      1
207260  207261      1
389769  389770      1
389768  389769      1
389767  389768      1


# Defining the Moods

### Mood Categories

The moods and categories we use are based on the mapping found in the paper ["Lyric Text Mining in Music Mood Classification"](http://www.ismir2009.ismir.net/proceedings/PS3-4.pdf).

H. Xiao, J. S. Downie, and A. F. Ehmann, “Lyric TextMining in Music Mood Classiﬁcation,” American Mu-sic, vol. 183, no. 5040, pp. 411–416, 2009



In [ ]:
from lastfm_labels import MOOD_CATEGORIES
print('Number of Mood Categories = {0}\n'.format(len(MOOD_CATEGORIES)))
print('Categories:\n')
pprint(MOOD_CATEGORIES)